In [1]:
from queue import Queue
from rtai.utils.config import YamlLoader
from rtai.llm.llm_client import LLMClient
from rtai.agent.agent_manager import AgentManager
from rtai.agent.agent import Agent
from rtai.world.world import World
from rtai.core.clock import clock

cfg = YamlLoader.load("configs/rtai_jason.yaml")
clock(cfg.expand('Clock'))
event_queue = Queue()

world = World(cfg.expand('World'), event_queue)

llm_client = LLMClient()
llm_client.initialize(cfg.expand('LLMClient'))

agent_mgr = AgentManager(event_queue, cfg.expand('Agents'), client=llm_client, world=world)
agent_mgr.initialize()
initial_shared_memories = world.get_shared_memories()
agent_mgr.load_initial_memories(initial_shared_memories)
agent_mgr.update(first_day=True)

agent1: Agent = agent_mgr.agents['Hank Thompson']
agent2: Agent = agent_mgr.agents['Claire Reynolds']

In [5]:
def pretty_print_schedule(name, daily_schedule):
    print(f"{name}'s Daily Schedule:")
    print("\n")
    for task in daily_schedule:
        print(task[0].split('\"')[0] + " for " + str(task[1]) + " hours starting at " + str(task[2]))

pretty_print_schedule(agent1.get_name(), agent1.s_mem.daily_schedule)
print("\n")
pretty_print_schedule(agent2.get_name(), agent2.s_mem.daily_schedule)


Hank Thompson's Daily Schedule:


Wakes up early, tends to crops, enjoys tractor rides, listens to country music, visits Claire for 2 hours starting at 8:00
Maintains crops, visits friend, plays country music for 3 hours starting at 1:00
Harvests crops for 4 hours starting at 6:00


Claire Reynolds's Daily Schedule:


Conduct field research on local ecosystems for 1 hours starting at 9:00
Analyze and report on findings from field data for 2 hours starting at 1:00
Present findings to colleagues and stakeholders for 3 hours starting at 9:00


In [9]:
# Dialogue
dialogue_topic = "Hank asking Claire to do an ecological survey of his farm"
out = llm_client.generate_dialogue(agent1, agent2, "Hank's Farm", dialogue_topic)
print(out)


Hank: Hey, Claire. I was thinking, you know a lot about ecology, right?
Claire: Yeah, I do. Why do you ask?

Hank: Well, I was wondering if you could do an ecological survey of my farm. You know, see how it's doing and if there's anything I can do to improve it.
Claire: Sure, I'd be happy to help. What kind of farm is it?

Hank: It's a corn and soybean farm. I've been doing it for years, but I just want to make sure I'm doing it right.
Claire: That's great. I'll make a note of that and get started on the survey.

Hank: Thanks, Claire. I really appreciate it.
Claire: No problem, Hank. I'm always happy to help out a friend.


ggml_metal_free: deallocating
